# Lecture 16: 2023-30-03 Vector Semantics II (cont.)

##  Lecture overview

- CBOW and Skip-gram analysis
- Using vectors to analyze sentence or document similarity

## CBOW and Skip-gram analysis

<img src="./images/mikolov.png" width="900" height="500" />

### CBOW



```python

import tensorflow as tf
from tqdm import tqdm

# Number of iterations to train the model
ITERATIONS = ...
# Tokenized text size
tokenized_text_size = ...
# Sliding window size for context words
WINDOW_SIZE = ...

# Start of the main training loop
for iter in tqdm(range(ITERATIONS)):
    # Variable to store the cumulative loss per epoch
    loss_per_epoch = 0

    # Sliding window over the tokenized text
    for start in range(tokenized_text_size - WINDOW_SIZE):
        # Extracting the window of words as indices
        indices = text_as_int[start:start + WINDOW_SIZE]

        # GradientTape is used to record the gradients during the forward pass
        with tf.GradientTape() as tape:
            # Variable to store the combined context vector
            combined_context = 0

            # Create a context slider by iterating through the indices
            for count, index in enumerate(indices):
                if count != WINDOW_SIZE // 2:
                    # Add the context word's vector to the combined context vector
                    combined_context += context_vector_matrix[index, :]

            # Normalize the combined context vector by dividing by the number of context words
            combined_context /= (WINDOW_SIZE - 1)

            # Calculate the dot product between the center vector matrix and the combined context vector
            output = tf.matmul(center_vector_matrix, tf.expand_dims(combined_context, 1))

            # Compute the softmax output for the center word
            softout = tf.nn.softmax(output, axis=0)
            # Calculate the loss for the center word
            loss = softout[indices[WINDOW_SIZE // 2]]

            # Calculate the log loss for the current window
            logloss = -tf.math.log(loss)

            # Update the cumulative loss for the current epoch
            loss_per_epoch += logloss.numpy()

            # Calculate the gradients for the context and center vector matrices
            grad = tape.gradient(logloss, [context_vector_matrix, center_vector_matrix])

            # Update the context and center vector matrices using the calculated gradients
            optimizer.apply_gradients(zip(grad, [context_vector_matrix, center_vector_matrix]))

    # Append the cumulative loss for the current epoch to the loss list
    loss_list.append(loss_per_epoch)

```

### [Gradient Tape](https://www.tensorflow.org/api_docs/python/tf/GradientTape)

In [3]:
import tensorflow as tf
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

x = tf.constant(3.0)
with tf.GradientTape() as g:
  g.watch(x)
  y = x * x
dy_dx = g.gradient(y, x)
print(dy_dx)


tf.Tensor(6.0, shape=(), dtype=float32)


### [Matmul](https://www.tensorflow.org/api_docs/python/tf/linalg/matmul)

In [22]:
# 2-D tensor
a = tf.constant([1, 2, 3, 4, 5, 6], shape=[2, 3])

# 2-D tensor
b = tf.constant([7, 8, 9, 10, 11, 12], shape=[3, 2])

c = tf.matmul(a, b)

d = tf.tensordot(a, b, axes=1)

print(a, b, c, d, sep='\n\n')

tf.Tensor(
[[1 2 3]
 [4 5 6]], shape=(2, 3), dtype=int32)

tf.Tensor(
[[ 7  8]
 [ 9 10]
 [11 12]], shape=(3, 2), dtype=int32)

tf.Tensor(
[[ 58  64]
 [139 154]], shape=(2, 2), dtype=int32)

tf.Tensor(
[[ 58  64]
 [139 154]], shape=(2, 2), dtype=int32)


### [Expand Dim](https://www.tensorflow.org/api_docs/python/tf/expand_dims)

In [10]:
image = tf.zeros([10,10,3])
image.shape

TensorShape([10, 10, 3])

In [12]:
expanded = tf.expand_dims(image, axis=0)
expanded.shape

TensorShape([1, 10, 10, 3])

In [13]:
image[0]

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

In [14]:
expanded[0]

<tf.Tensor: shape=(10, 10, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0.

### [Softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax)

In [19]:
softmax = tf.nn.softmax([-1, 0., 1.])
softmax


<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

In [20]:
sum(softmax)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

## Skip-gram

```python

import tensorflow as tf
from tqdm import tqdm

# Number of iterations to train the model
ITERATIONS = ...
# Tokenized text size
tokenize_text_size = ...
# Sliding window size for context words
WINDOW_SIZE = ...

# Start of the main training loop
for iter in tqdm(range(ITERATIONS)):
    # Variable to store the cumulative loss per epoch
    loss_per_epoch = 0

    # Sliding window over the tokenized text
    for start in range(tokenize_text_size - WINDOW_SIZE):
        # Extracting the window of words as indices
        indices = text_as_int[start:start + WINDOW_SIZE]

        # GradientTape is used to record the gradients during the forward pass
        with tf.GradientTape() as tape:
            # Initialize the loss for this window to 0
            loss = 0

            # Extract the center word's vector from the center vector matrix
            center_vector = center_vector_matrix[indices[WINDOW_SIZE // 2], :]
            # Calculate the dot product between context vector matrix and center vector
            output = tf.matmul(context_vector_matrix, tf.expand_dims(center_vector, 1))

            # Compute the softmax output for each context word
            softmax_output = tf.nn.softmax(output, axis=0)

            # Calculate the loss for each context word in the window
            for (count, index) in enumerate(indices):
                if count != WINDOW_SIZE // 2:
                    loss += softmax_output[index]

            # Calculate the log loss for the current window
            logloss = -tf.math.log(loss)

            # Update the cumulative loss for the current epoch
            loss_per_epoch += logloss.numpy()

            # Calculate the gradients for the context and center vector matrices
            grad = tape.gradient(logloss, [context_vector_matrix, center_vector_matrix])

            # Update the context and center vector matrices using the calculated gradients
            optimizer.apply_gradients(zip(grad, [context_vector_matrix, center_vector_matrix]))

    # Append the cumulative loss for the current epoch to the loss list
    loss_list.append(loss_per_epoch)


```



## Using vectors to analyze sentence or document similarity

### Dot product

The dot product or inner product of two vectors is defined as:

$$ \vec{a} \cdot \vec{b} = \sum_{i=1}^{n} a_i b_i $$


With our vectors are defined as:

$$ |v| = \sqrt{\sum^{N}_{i=1} v^2_i}$$

* The longer the vector, the larger the magnitude
* More frequent words will have larger magnitude
* Raw dot product is not normalized - how can we use it to measure similarity?


### Normalized dot product

$$ \vec{a} \cdot \vec{b} = \frac{\sum_{i=1}^{n} a_i b_i}{\sqrt{\sum^{N}_{i=1} a^2_i} \sqrt{\sum^{N}_{i=1} b^2_i}} $$

### Cosine similarity

With the cosine similarity, we can measure the angle between two vectors. The cosine similarity is defined as:

$$ \text{cosine(a, b)  = } \frac{a\cdot b}{|a||b|} = \frac{\sum_{i=1}^{n} a_i b_i}{\sqrt{\sum^{N}_{i=1} a^2_i} \sqrt{\sum^{N}_{i=1} b^2_i}} $$

### Cosine similarity of words

https://www.tensorflow.org/tensorboard/tensorboard_projector_plugin

### Cosine similarity of sentences and documents

How can we use the word vectors to measure the similarity between sentences or documents?

* Average the word vectors in the sentence or document
* Calculate the cosine similarity between the two sentences or documents
* Train a classifier to predict the similarity between sentences or documents
* Train a sentence embedding model to generate sentence or document vectors
* etc.


#### Doc2Vec

https://radimrehurek.com/gensim/models/doc2vec.html


In [23]:
import gensim

from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(common_texts)]
model = Doc2Vec(documents, vector_size=5, window=2, min_count=1, workers=4)

In [24]:
Doc2Vec??

Init signature:
Doc2Vec(
    documents=None,
    corpus_file=None,
    vector_size=100,
    dm_mean=None,
    dm=1,
    dbow_words=0,
    dm_concat=0,
    dm_tag_count=1,
    dv=None,
    dv_mapfile=None,
    comment=None,
    trim_rule=None,
    callbacks=(),
    window=5,
    epochs=10,
    shrink_windows=True,
    **kwargs,
)
Docstring:     
Serialize/deserialize objects from disk, by equipping them with the `save()` / `load()` methods.

Warnings
--------
This uses pickle internally (among other techniques), so objects must not contain unpicklable attributes
such as lambda functions etc.
Source:        
class Doc2Vec(Word2Vec):
    def __init__(
            self, documents=None, corpus_file=None, vector_size=100, dm_mean=None, dm=1, dbow_words=0, dm_concat=0,
            dm_tag_count=1, dv=None, dv_mapfile=None, comment=None, trim_rule=None, callbacks=(),
            window=5, epochs=10, shrink_windows=True, **kwargs,
        ):
        """Class for training, using and evaluating ne

In [27]:
common_texts

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]

In [26]:
model.infer_vector(["system", "response"])

array([-0.00504203,  0.0676716 , -0.07490709,  0.01672102,  0.09844229],
      dtype=float32)

### Let's train a Doc2Vec model


In [28]:
import os
import re
import pandas as pd
from gensim import corpora, models, similarities

df = pd.read_csv('./data/IMDB_Dataset.csv')

df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [37]:
pos_reviews = df[df['sentiment'] == 'positive']
neg_reviews = df[df['sentiment'] == 'negative']

pos_reviews = pos_reviews['review'].tolist()
neg_reviews = neg_reviews['review'].tolist()

In [46]:
# Define a function to clean up the reviews
def clean_review(review):
    review = review.lower()  # Convert to lowercase
    review = re.sub(r"<[^>]*>", "", review)  # Remove HTML tags
    review = re.sub(r"[^a-z0-9]+", " ", review)  # Remove non-alphanumeric characters
    return review.strip()


# Combine the positive and negative reviews into a single list
train_reviews = pos_reviews[:100] + neg_reviews[:100]

# Examine the first review
train_reviews[0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [47]:
# Create a Gensim dictionary from the reviews
dictionary = corpora.Dictionary([review.split() for review in train_reviews])

In [73]:
dictionary??

Type:           Dictionary
String form:    Dictionary<11492 unique tokens: ['(crooked', '/><br', '/>I', '/>It', '/>The']...>
Length:         11492
File:           /media/james/Projects/GitHub/DATA_340_NLP/Notebooks/venv/lib/python3.10/site-packages/gensim/corpora/dictionary.py
Source:        
class Dictionary(utils.SaveLoad, Mapping):
    """Dictionary encapsulates the mapping between normalized words and their integer ids.

    Notable instance attributes:

    Attributes
    ----------
    token2id : dict of (str, int)
        token -> token_id. I.e. the reverse mapping to `self[token_id]`.
    cfs : dict of (int, int)
        Collection frequencies: token_id -> how many instances of this token are contained in the documents.
    dfs : dict of (int, int)
        Document frequencies: token_id -> how many documents contain this token.
    num_docs : int
        Number of documents processed.
    num_pos : int
        Total number of corpus positions (number of processed words).
    nu

In [49]:
# Convert the reviews into Gensim bag-of-words vectors
corpus = [dictionary.doc2bow(review.split()) for review in train_reviews]

In [74]:
corpus??

Type:        list
String form: [[(0, 1), (1, 3), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 5), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 3), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 6), (44, 1), (45, 2), (46, 1), (47, 4), (48, 1), (49, 1), (50, 1), (51, 2), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 2), (74, 2), (75, 1), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 2), (84, 1), (85, 5), (86, 2), (87, 1), (88, 1), (89, 1), (90, 2), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 2), (103, 1), (104, 1), (105, 1), (106, 3), (107, 1

In [50]:
# Train a Gensim TF-IDF model on the corpus
tfidf = models.TfidfModel(corpus)

In [76]:
tfidf?

Type:           TfidfModel
String form:    TfidfModel<num_docs=200, num_nnz=30991>
File:           /media/james/Projects/GitHub/DATA_340_NLP/Notebooks/venv/lib/python3.10/site-packages/gensim/models/tfidfmodel.py
Docstring:     
Objects of this class realize the transformation between word-document co-occurrence matrix (int)
into a locally/globally weighted TF-IDF matrix (positive floats).

Examples
--------
.. sourcecode:: pycon

    >>> import gensim.downloader as api
    >>> from gensim.models import TfidfModel
    >>> from gensim.corpora import Dictionary
    >>>
    >>> dataset = api.load("text8")
    >>> dct = Dictionary(dataset)  # fit dictionary
    >>> corpus = [dct.doc2bow(line) for line in dataset]  # convert corpus to BoW format
    >>>
    >>> model = TfidfModel(corpus)  # fit model
    >>> vector = model[corpus[0]]  # apply model to the first corpus document
Init docstring:
Compute TF-IDF by multiplying a local component (term frequency) with a global component
(inverse d

In [51]:
# Convert the corpus into Gensim TF-IDF vectors
tfidf_corpus = tfidf[corpus]

In [77]:
tfidf_corpus?

Type:           TransformedCorpus
String form:    <gensim.interfaces.TransformedCorpus object at 0x7fa6565dd930>
Length:         200
File:           /media/james/Projects/GitHub/DATA_340_NLP/Notebooks/venv/lib/python3.10/site-packages/gensim/interfaces.py
Docstring:      Interface for corpora that are the result of an online (streamed) transformation.
Init docstring:
Parameters
----------
obj : object
    A transformation :class:`~gensim.interfaces.TransformationABC` object that will be applied
    to each document from `corpus` during iteration.
corpus : iterable of list of (int, number)
    Corpus in bag-of-words format.
chunksize : int, optional
    If provided, a slightly more effective processing will be performed by grouping documents from `corpus`.

In [101]:
# Train a Gensim LSI model on the TF-IDF vectors
lsi = models.LsiModel(tfidf_corpus, id2word=dictionary, num_topics=25)

In [102]:
lsi??

Type:           LsiModel
String form:    LsiModel<num_terms=11492, num_topics=25, decay=1.0, chunksize=20000>
File:           /media/james/Projects/GitHub/DATA_340_NLP/Notebooks/venv/lib/python3.10/site-packages/gensim/models/lsimodel.py
Source:        
class LsiModel(interfaces.TransformationABC, basemodel.BaseTopicModel):
    """Model for `Latent Semantic Indexing
    <https://en.wikipedia.org/wiki/Latent_semantic_analysis#Latent_semantic_indexing>`_.

    The decomposition algorithm is described in `"Fast and Faster: A Comparison of Two Streamed
    Matrix Decomposition Algorithms" <https://arxiv.org/pdf/1102.5597.pdf>`_.

    Notes
    -----
    * :attr:`gensim.models.lsimodel.LsiModel.projection.u` - left singular vectors,
    * :attr:`gensim.models.lsimodel.LsiModel.projection.s` - singular values,
    * ``model[training_corpus]`` - right singular vectors (can be reconstructed if needed).

    See Also
    --------
    `FAQ about LSI matrices
    <https://github.com/RaRe-Technolo

In [103]:
# Convert the corpus into Gensim LSI vectors
lsi_corpus = lsi[tfidf_corpus]

In [104]:
# Set up a Gensim index for similarity search
index_lsi = similarities.MatrixSimilarity(lsi_corpus)

In [105]:
# Define a function to find the most similar reviews
def find_similar_reviews(review_text, num_results=10):
    review_text = clean_review(review_text)
    vec_bow = dictionary.doc2bow(review_text.split())
    vec_tfidf = tfidf[vec_bow]
    vec_lsi = lsi[vec_tfidf]
    sims = index_lsi[vec_lsi]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    
    results = [] 
    for i in range(num_results):
        index, score = sims[i]
        results.append((train_reviews[index], score))
    return results

In [106]:
# Test the function with a sample review
from IPython.display import HTML, Markdown

review = "The movie was a work of art in the world of motion pictures. The filmmaking was exceptional and the acting was remarkable. The storyline was complex yet captivating, and the special effects were breathtaking."
similar_reviews = find_similar_reviews(review)

for i, (review_text, score) in enumerate(similar_reviews):
    print(f"Similarity score for review {i+1}: {score}")
    print(display(Markdown(f"<p>{review_text}</p>")))

Similarity score for review 1: 0.6704679727554321


<p>This was probably the worst movie i have ever seen in my life!! It was stupid there was no plot and the special affects were ridiculous!! And i have never seen such bad acting in my life! The only good part about the movie were all the hot guys(especially Drew Fuller). I don't know what these people were thinking when they made this movie!! I didn't even want to finish the whole thing because you get to this point in the movie where the guys are all in bed touching themselves. I mean it was like some kind of sick and twisted kiddy porn! I would advise anyone who has heard of this movie and was interested in seeing it to just forget about it and find another movie to watch! I was very disappointed!! The whole movie was a complete waste of time in my opinion.</p>

None
Similarity score for review 2: 0.6045014262199402


<p>What seemed at first just another introverted French flick offering no more than baleful sentiment became for me, on second viewing, a genuinely insightful and quite satisfying presentation.<br /><br />Spoiler of sorts follows.<br /><br />Poor Cedric; he apparently didn't know what hit him. Poor audience; we were at first caught up in what seemed a really beautiful and romantic story only to be led back and forth into the dark reality of mismatch. These two guys just didn't belong together from their first ambiguous encounter. As much as Mathieu and Cedric were sexually attracted to each other, the absence of a deeper emotional tie made it impossible for Mathieu, an intellectual being, to find fulfillment in sharing life with someone whose sensibilities were more attuned to carnival festivities and romps on the beach.<br /><br />On a purely technical note, I loved the camera action in this film. Subtitles were totally unnecessary, even though my French is "presque rien." I could watch it again without the annoying English translation and enjoy it even more. This was a polished, very professionally made motion picture. Though many scenes seem superfluous, I rate it nine out of ten.</p>

None
Similarity score for review 3: 0.5853372812271118


...

None
Similarity score for review 9: 0.5431765913963318


<p>This movie was not very well directed. they almost totally disregarded the book.I guess they were trying 2 save time. the only upside 2 me was that the actor who played finny was cute. Some of the dialog between the main characters appeared a little gay which was not the case in the book. Major parts of the book were once again chopped out.You lost the over all effect it was not as haunting as the book and left me lacking severely. Also the strong language although it was brief was very unnecessary. Also i was surprised ( not pleasantly) by a new character that was no where in the book.One of my favorite characters (leper) was poorly interpreted and portrayed. He seemed more sinister in the movie than the real leper was in the book. Over all disappointing.</p>

None
Similarity score for review 10: 0.5289626717567444


<p>I haven't read the Anne Rice novel that this movie was based on, but who knows, maybe reading the book is cheaper than renting QUEEN OF THE DAMNED and is probably better for your health. It isn't that this movie is necessarily bad for your health, but a book can be very relaxing and certainly exercises the active part of your brain more so than this movie. You can count the number of pages by Anne Rice that I've read on one hand, but after seeing this movie and Interview with a Vampire, I get the feeling that she writes really good novels. The plots for both movies hint at a whole sea of deep and interwoven vampire history.<br /><br />Still, Stuart Townsend's voice-over narration gets a heck of a lot more annoying than Brad Pitt's vampire narrative ever did, and you can tell that QUEEN OF THE DAMNED's limited production resources barely give enough flesh to the Anne Rice storyline. While Interview decided to go with lace and elegance, QUEEN relies on low budget special effects that try really hard to be taken seriously. One can see that the original novel had potential as a movie and that the production team focused its attention in the wrong places. The costumes and rock & roll stage could have been replaced with more blood and an eerier soundtrack.<br /><br />However, I'll give credit where credit is due. The soundtrack is excellent. Korn and Disturbed had me down with the sickness bobbing my noggin like Butthead.<br /><br />The film opens with a very cool Goth-rock zoom & splice montage, but after the first ten minutes or so, the directing degenerates quickly. It's as if the movie was so long that the director realized that there wasn't enough time and enough money to do an Anne Rice novel justice. What results are some mediocre vampire scenes and plenty of cheesy special effects. Unfortunately, QUEEN OF THE DAMNED fails to do the genre justice just as its John Carpenter counterparts fail to impress. Where are the yellow contacts? Where's the pale blue make-up? Scene after scene, I shook my head reminiscing about the days of Salem's Lot and Fright Night when low budget was done right.<br /><br />There are redeeming qualities though that save this movie from being garbage. Props to Aaliyah, and may her soul forever rest in peace. She might have become a renowned actress, had her life not been taken from us so prematurely, for she did give this movie a decent performance with plenty of nice belly dancing. Did I mention that the soundtrack was good? Let's see, what else can I say? It wasn't too long. The Anne Rice novel could have easily been a three hour movie if an ambitious director like Francis Ford Coppola got his hands on it. There are a few twists and turns here and there in the plot. But all in all it was a legitimate rock and roll addition to the slew of second-rate vampire movies out there. The director of this movie went on to direct a new Battlestar Galactica mini series if that tells you anything.<br /><br />JY<br /><br />Jimboduck-dot-com</p>

None
